In [2]:
import pandas as pd
import numpy as np

In [3]:
raw_data=pd.read_csv('D:/PICT/BE/Semester-1/LP-4/credit_card/creditcard.csv')

In [4]:
raw_data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [5]:
targets, frequencies = np.unique(raw_data.iloc[:,-1],return_counts=True)

In [6]:
print(targets)

[0 1]


In [7]:
print(frequencies)

[284315    492]


In [8]:
inputs=raw_data.iloc[:,:-1]
labels=raw_data.iloc[:,-1]

In [9]:
inputs.shape

(284807, 30)

In [10]:
labels.shape

(284807,)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
x_train, x_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.2, stratify=labels)

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [14]:
min_max_scaler=MinMaxScaler(feature_range=(0,1))

In [15]:
x_train_scaled= min_max_scaler.fit_transform(x_train.copy())
x_test_scaled= min_max_scaler.transform(x_test.copy())

In [16]:
x_test_scaled.shape[1]

30

In [17]:
import tensorflow as tf

from tensorflow import keras

from keras.models import Sequential

from keras import Model

from keras.layers import Dense,Dropout

In [22]:
class AutoEncoder(tf.keras.Model):

    def __init__(self,output_units,code_size=8):
        super().__init__()
        self.encoder = tf.keras.models.Sequential(
            [
                Dense(64,activation='relu'), 
                Dropout(0.1), 
                Dense(32,activation='relu'), 
                Dropout(0.1), 
                Dense(16,activation='relu'),
                Dropout(0.1), 
                Dense(code_size,activation='relu')
            ]
        )

        self.decoder = tf.keras.models.Sequential([
            Dense(16,activation='relu'),
            Dropout(0.1), 
            Dense(32,activation='relu'), 
            Dropout(0.1), 
            Dense(64,activation='relu'),
            Dropout(0.1), 
            Dense(output_units,activation='sigmoid') 
        ])

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [23]:
model=AutoEncoder(output_units=x_train_scaled.shape[1])

In [24]:
model.compile(loss='msle',optimizer='adam',metrics=['mse'])

In [25]:
model.fit(x_train_scaled,y_train,epochs=10,validation_data=(x_test_scaled,y_test),batch_size=512)

Epoch 1/10
433/446 [============================>.] - ETA: 0s - loss: 0.0077 - mse: 0.0118

KeyboardInterrupt: 

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential

In [3]:
raw_data=pd.read_csv('D:/PICT/BE/Semester-1/LP-4/credit_card/creditcard.csv')
raw_data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [5]:
features=raw_data.iloc[:,:-1]
target=raw_data.iloc[:,-1]
target

0         0
1         0
2         0
3         0
4         0
         ..
284802    0
284803    0
284804    0
284805    0
284806    0
Name: Class, Length: 284807, dtype: int64

In [14]:
X_train,X_test,y_test,y_train=train_test_split(features,target,test_size=0.2)
scaler=MinMaxScaler(feature_range=(0,1))
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.fit_transform(X_test)


In [22]:
class Autoencoder(tf.keras.Model):
    def __init__(self,output_units,code_size=8):
        super().__init__()
        self.encoder=Sequential([
            Dense(64,activation="relu"),
            Dropout(0.01),
            Dense(32,activation="relu"),
            Dropout(0.01),
            Dense(16,activation="relu"),
            Dropout(0.01),
            Dense(code_size,activation="relu"),
        ])
        self.decoder=Sequential([
            Dense(16,activation="relu"),
            Dropout(0.01),
            Dense(32,activation="relu"),
            Dropout(0.01),
            Dense(64,activation="relu"),
            Dropout(0.01),
            Dense(output_units,activation="sigmoid"),
        ])
    def call(self,inputs):
        encoded=self.encoder(inputs)
        decoded=self.decoder(encoded)
        return decoded
        

In [23]:
model=Autoencoder(output_units=X_train_scaled.shape[1])
model.compile(loss="msle",optimizer="adam",metrics=["mse"])

In [24]:
history=model.fit(X_train_scaled,y_test,epochs=1,batch_size=64)

3561/3561 [==============================] - 6s 2ms/step - loss: 0.0018 - mse: 0.0031


In [25]:
model.evaluate(X_test_scaled,y_train)

1781/1781 [==============================] - 1s 720us/step - loss: 6.4499e-04 - mse: 0.0014


[0.0006449927459470928, 0.0013945546234026551]